In [2]:
# Block 1: Import necessary libraries
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle
from pathlib import Path
import pandas as pd

In [3]:
class Config:
    # Image preprocessing parameters
    IMAGE_SIZE = (32, 32)  # As mentioned in paper: 32x32 pixels
    FEATURE_SIZE = 1024   # 32*32 = 1024 pixel values as features

    # KNN parameters
    K_NEIGHBORS = 3  # As specified in the paper

    # File paths (adjust these to your data structure)
    DATA_PATHS = {
        'digits': 'path/to/digit_images/',
        'keywords': 'path/to/keyword_images/',
        'alphabets': 'path/to/alphabet_images/',
    }

    # Bangladeshi characters as per BRTA regulations (41 possible alphabets)
    BRTA_ALPHABETS = [
        'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ',
        'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন',
        'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ',
        'স', 'হ', 'ড়', 'ঢ়', 'য়', 'ৎ', 'ং', 'ঃ', 'ঁ', '্', 'া'
    ]

    # Common keywords in Bangladeshi license plates
    KEYWORDS = ['ঢাকা', 'মেট্রো', 'পুর', 'খোলী', 'চট্ট', 'সিলেট', 'রাজ', 'বরি']

    # Digits 0-9 in Bengali
    BENGALI_DIGITS = ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']

config = Config()

In [4]:
def preprocess_image(image_path):
    """
    Preprocess image according to the paper's methodology:
    1. Convert to grayscale using Y = 0.21R + 0.72G + 0.07B
    2. Resize to 32x32
    3. Normalize pixel values
    """
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        return None

    # Convert to grayscale using the formula from paper
    if len(img.shape) == 3:
        gray = 0.21 * img[:,:,2] + 0.72 * img[:,:,1] + 0.07 * img[:,:,0]
        gray = gray.astype(np.uint8)
    else:
        gray = img

    # Resize to fixed size (32x32 as mentioned in paper)
    resized = cv2.resize(gray, config.IMAGE_SIZE)

    # Normalize pixel values to 0-1 range
    normalized = resized.astype(np.float32) / 255.0

    return normalized

def extract_features(image):
    """
    Extract features from preprocessed image.
    As per paper: pixel intensity values are used as features
    """
    if image is None:
        return None

    # Flatten the 32x32 image to 1024 feature vector
    features = image.flatten()
    return features

def apply_morphological_operations(image):
    """
    Apply morphological operations (erosion and dilation) as mentioned in paper
    """
    kernel = np.ones((3,3), np.uint8)

    # Erosion followed by dilation (opening operation)
    eroded = cv2.erode(image, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)

    return dilated

print("Preprocessing functions defined successfully!")

Preprocessing functions defined successfully!


In [6]:
class DataLoader:
    def __init__(self, config):
        self.config = config
        self.features = []
        self.labels = []
        self.label_to_class = {}
        self.class_to_label = {}

    def load_images_from_directory(self, directory, class_name):
        """Load images from a directory and assign them a class label"""
        if not os.path.exists(directory):
            print(f"Warning: Directory {directory} not found!")
            return

        image_files = [f for f in os.listdir(directory)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

        print(f"Loading {len(image_files)} images from {directory} for class '{class_name}'")

        for img_file in image_files:
            img_path = os.path.join(directory, img_file)

            # Preprocess image
            processed_img = preprocess_image(img_path)
            if processed_img is None:
                continue

            # Extract features
            features = extract_features(processed_img)
            if features is None:
                continue

            self.features.append(features)
            self.labels.append(class_name)

    def prepare_datasets(self):
        """Prepare the complete dataset"""
        print("Starting data preparation...")

        # Load digits (0-9)
        for i, digit in enumerate(config.BENGALI_DIGITS):
            digit_dir = os.path.join(config.DATA_PATHS['digits'], str(i))
            self.load_images_from_directory(digit_dir, f"digit_{digit}")

        # Load keywords
        for keyword in config.KEYWORDS:
            keyword_dir = os.path.join(config.DATA_PATHS['keywords'], keyword)
            self.load_images_from_directory(keyword_dir, f"keyword_{keyword}")

        # Load alphabets (41 BRTA alphabets)
        for alphabet in config.BRTA_ALPHABETS:
            alphabet_dir = os.path.join(config.DATA_PATHS['alphabets'], alphabet)
            self.load_images_from_directory(alphabet_dir, f"alphabet_{alphabet}")

        # Convert to numpy arrays
        self.features = np.array(self.features)
        self.labels = np.array(self.labels)

        # Create label mappings
        unique_labels = np.unique(self.labels)
        self.label_to_class = {label: i for i, label in enumerate(unique_labels)}
        self.class_to_label = {i: label for i, label in enumerate(unique_labels)}

        # Convert string labels to numeric
        numeric_labels = np.array([self.label_to_class[label] for label in self.labels])

        print(f"Dataset prepared successfully!")
        print(f"Total samples: {len(self.features)}")
        print(f"Feature dimension: {self.features.shape[1]}")
        print(f"Number of classes: {len(unique_labels)}")

        return self.features, numeric_labels

data_loader = DataLoader(config)
print("Data loader initialized!")

Data loader initialized!


In [7]:
# Block 5: Model Training (KNN with k=3)
class BanglaCharacterRecognizer:
    def __init__(self, k_neighbors=3):
        self.k_neighbors = k_neighbors
        self.model = KNeighborsClassifier(n_neighbors=k_neighbors,
                                        metric='euclidean',
                                        weights='uniform')
        self.is_trained = False
        self.label_mappings = {}

    def train(self, X_train, y_train, label_mappings):
        """Train the KNN model"""
        print(f"Training KNN model with k={self.k_neighbors}...")

        self.model.fit(X_train, y_train)
        self.label_mappings = label_mappings
        self.is_trained = True

        print("Model training completed!")

    def predict(self, features):
        """Predict character class for given features"""
        if not self.is_trained:
            raise ValueError("Model not trained yet!")

        # Ensure features is 2D
        if len(features.shape) == 1:
            features = features.reshape(1, -1)

        predictions = self.model.predict(features)
        probabilities = self.model.predict_proba(features)

        return predictions, probabilities

    def predict_character(self, image_path):
        """Predict character from image path"""
        # Preprocess image
        processed_img = preprocess_image(image_path)
        if processed_img is None:
            return None, None

        # Extract features
        features = extract_features(processed_img)
        if features is None:
            return None, None

        # Predict
        prediction, probability = self.predict(features)

        # Convert numeric prediction back to character
        class_name = self.label_mappings['class_to_label'][prediction[0]]
        confidence = np.max(probability[0])

        return class_name, confidence

    def save_model(self, filepath):
        """Save trained model to disk"""
        model_data = {
            'model': self.model,
            'k_neighbors': self.k_neighbors,
            'label_mappings': self.label_mappings,
            'is_trained': self.is_trained
        }

        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath):
        """Load trained model from disk"""
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)

        self.model = model_data['model']
        self.k_neighbors = model_data['k_neighbors']
        self.label_mappings = model_data['label_mappings']
        self.is_trained = model_data['is_trained']

        print(f"Model loaded from {filepath}")

# Initialize the recognizer
recognizer = BanglaCharacterRecognizer(k_neighbors=config.K_NEIGHBORS)
print("Character recognizer initialized!")

Character recognizer initialized!


In [ ]:
# Block 6: Training Pipeline
def train_model():
    """Complete training pipeline"""
    print("=" * 50)
    print("STARTING TRAINING PIPELINE")
    print("=" * 50)

    # Prepare dataset
    features, labels = data_loader.prepare_datasets()

    if len(features) == 0:
        print("No data found! Please check your data paths.")
        return None

    # Prepare label mappings
    label_mappings = {
        'label_to_class': data_loader.label_to_class,
        'class_to_label': data_loader.class_to_label
    }

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.2, random_state=42, stratify=labels
    )

    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")

    # Train the model
    recognizer.train(X_train, y_train, label_mappings)

    # Evaluate the model
    print("\nEvaluating model...")
    train_predictions, _ = recognizer.predict(X_train)
    test_predictions, _ = recognizer.predict(X_test)

    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

    # Detailed classification report
    print("\nDetailed Classification Report:")
    print(classification_report(y_test, test_predictions,
                              target_names=[label_mappings['class_to_label'][i]
                                          for i in range(len(label_mappings['class_to_label']))]))

    return recognizer

# Uncomment the next line to run training (when you have data)
# trained_model = train_model()

In [ ]:
# Block 7: Inference and Testing Functions
def test_single_image(image_path, model):
    """Test the model on a single image"""
    if not model.is_trained:
        print("Model not trained yet!")
        return

    print(f"Testing image: {image_path}")

    # Predict character
    predicted_class, confidence = model.predict_character(image_path)

    if predicted_class is None:
        print("Failed to process image!")
        return

    print(f"Predicted: {predicted_class}")
    print(f"Confidence: {confidence:.4f}")

    # Display the image
    img = cv2.imread(image_path)
    if img is not None:
        plt.figure(figsize=(6, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Original Image")
        plt.axis('off')

        # Show preprocessed version
        processed = preprocess_image(image_path)
        if processed is not None:
            plt.subplot(1, 2, 2)
            plt.imshow(processed, cmap='gray')
            plt.title(f"Processed (32x32)\nPredicted: {predicted_class}\nConf: {confidence:.3f}")
            plt.axis('off')

        plt.tight_layout()
        plt.show()

def batch_test_images(image_directory, model):
    """Test the model on a batch of images"""
    if not model.is_trained:
        print("Model not trained yet!")
        return

    if not os.path.exists(image_directory):
        print(f"Directory {image_directory} not found!")
        return

    image_files = [f for f in os.listdir(image_directory)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    results = []

    print(f"Testing {len(image_files)} images...")

    for img_file in image_files:
        img_path = os.path.join(image_directory, img_file)
        predicted_class, confidence = model.predict_character(img_path)

        results.append({
            'filename': img_file,
            'predicted_class': predicted_class,
            'confidence': confidence
        })

    # Create results DataFrame
    results_df = pd.DataFrame(results)
    print("\nBatch Testing Results:")
    print(results_df.head(10))

    return results_df

In [ ]:
# Block 8: Utility Functions for License Plate Processing
def segment_license_plate_characters(plate_image_path):
    """
    Segment individual characters from a license plate image
    This implements the segmentation logic from the paper
    """
    img = cv2.imread(plate_image_path)
    if img is None:
        return []

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Apply morphological operations
    kernel = np.ones((3,3), np.uint8)
    processed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area and aspect ratio
    character_contours = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 100:  # Minimum area threshold
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / h

            # Filter based on typical character aspect ratios
            if 0.2 < aspect_ratio < 2.0:
                character_contours.append((x, y, w, h))

    # Sort contours from left to right
    character_contours.sort(key=lambda x: x[0])

    # Extract character images
    character_images = []
    for x, y, w, h in character_contours:
        char_img = gray[y:y+h, x:x+w]

        # Resize to standard size
        char_img_resized = cv2.resize(char_img, config.IMAGE_SIZE)
        character_images.append(char_img_resized)

    return character_images

def recognize_license_plate(plate_image_path, model):
    """
    Complete license plate recognition pipeline
    """
    if not model.is_trained:
        print("Model not trained yet!")
        return None

    print(f"Processing license plate: {plate_image_path}")

    # Segment characters
    character_images = segment_license_plate_characters(plate_image_path)

    if not character_images:
        print("No characters found in the image!")
        return None

    print(f"Found {len(character_images)} characters")

    # Recognize each character
    recognized_text = []
    confidences = []

    for i, char_img in enumerate(character_images):
        # Normalize the character image
        char_img_norm = char_img.astype(np.float32) / 255.0
        features = char_img_norm.flatten()

        # Predict character
        prediction, probability = model.predict(features)

        # Get character class and confidence
        class_name = model.label_mappings['class_to_label'][prediction[0]]
        confidence = np.max(probability[0])

        recognized_text.append(class_name)
        confidences.append(confidence)

        print(f"Character {i+1}: {class_name} (confidence: {confidence:.3f})")

    # Combine results
    full_text = ''.join([text.split('_')[1] if '_' in text else text for text in recognized_text])
    avg_confidence = np.mean(confidences)

    print(f"Recognized License Plate: {full_text}")
    print(f"Average Confidence: {avg_confidence:.3f}")

    return {
        'text': full_text,
        'individual_characters': recognized_text,
        'confidences': confidences,
        'average_confidence': avg_confidence
    }

print("All functions defined successfully!")
print("\n" + "="*60)
print("IMPLEMENTATION COMPLETE!")
print("="*60)
print("\nTo use this code:")
print("1. Update the data paths in Config class")
print("2. Organize your data in the specified directory structure")
print("3. Run train_model() to train the KNN classifier")
print("4. Use test_single_image() or recognize_license_plate() for inference")
print("\nExample usage:")
print("# Train model")
print("trained_model = train_model()")
print("\n# Test single character")
print("test_single_image('path/to/character.jpg', trained_model)")
print("\n# Recognize full license plate")
print("result = recognize_license_plate('path/to/license_plate.jpg', trained_model)")